In [1]:
import pandas as pd
import numpy as np

In [2]:
# Get risk free rate
predictor_raw =  pd.read_excel(open('ml_equity_premium_data.xlsx', 'rb'), sheet_name='PredictorData1926-2020')
predictor_raw.set_index(keys='yyyymm', inplace=True)
n_rows = predictor_raw.shape[0]
risk_free_lag = predictor_raw['Rfree'][0:(n_rows - 1)].loc[195701:].to_numpy()

In [3]:
# Get actual simple returns
predictor_df = pd.read_excel(open('ml_equity_premium_data.xlsx', 'rb'), sheet_name='result_predictor')
predictor_df.set_index(keys='month', inplace=True)
actual_simple_returns = predictor_df.equity_premium.loc[195702:202012].to_numpy() + risk_free_lag
# Historical average forecasting as benchmark
y_pred_HA = predictor_df['log_equity_premium'].iloc[0:(predictor_df.shape[0] - 1), ].cumsum() / np.arange(1, predictor_df.shape[0])
y_pred_HA = y_pred_HA.loc[195701:].to_numpy()

In [4]:
# Get the predicted log equity premiums and covert them into simple returns
y_ml_pred = pd.read_excel(open("ml_equity_premium_results.xlsx", 'rb'), sheet_name='ml_pred_equity_premium', index_col=[0])
# select the best 8 ML models based on oos_r_square
selected_ml_models = ['PLS', 'PCR', 'LASSO', 'ENet', 'RF', 'NN2', 'NN4', 'Ridge']
y_ml_pred = y_ml_pred[selected_ml_models]
# Add HA forecast
y_ml_pred['HA'] = y_pred_HA
# convert log equity premium to simple equity premium
y_ml_pred_simple_returns = pd.DataFrame()
for col in y_ml_pred.columns:
    y_ml_pred_simple_returns[col] = np.exp(y_ml_pred[col].to_numpy()) * (1 + risk_free_lag) - 1

# Add HA forecast
y_ml_pred['HA'] = y_pred_HA
y_ml_pred_simple_returns.head()

,PLS,PCR,LASSO,ENet,RF,NN2,NN4,Ridge,HA
0,-0.013874,0.009959,0.009212,0.009212,0.015038,0.012516,0.017810,0.009212,0.009178
1,-0.013948,0.010353,0.008910,0.008910,0.013908,0.009908,0.016719,0.008910,0.008780
2,-0.012376,0.010433,0.008809,0.008809,0.013806,0.008903,0.016521,0.008809,0.008721
3,-0.005925,0.011860,0.009011,0.009011,0.013413,0.006252,0.016718,0.009011,0.009024
4,0.006284,0.012422,0.009111,0.009111,0.014937,0.008069,0.017079,0.009111,0.009205


In [5]:
# Get market timing performance for long-short strategy
# inert the performance of buy & hold strategy
buy_hold_avg_return = np.mean(actual_simple_returns) * 12 * 100
buy_hold_Sharpe_ratio = np.mean(actual_simple_returns - risk_free_lag) / np.std(actual_simple_returns  - risk_free_lag)
market_timing_perf_df = pd.DataFrame(np.array([buy_hold_avg_return, buy_hold_Sharpe_ratio]).reshape(1, -1), index=['Buy-and-hold'],
                                     columns=["Average return(%)", "Sharpe ratio"])
for col in y_ml_pred.columns:
    pred_simple_returns = y_ml_pred_simple_returns[col].to_numpy()
    timing_signal = pred_simple_returns * actual_simple_returns > 0
    return_portfolio = actual_simple_returns * timing_signal * 12
    avg_return = np.mean(return_portfolio)
    excess_return_portfolio = return_portfolio - risk_free_lag
    SharpeRatio = np.mean(excess_return_portfolio) / np.std(excess_return_portfolio)
    market_timing_perf_df.loc[col] = [avg_return * 100, SharpeRatio]

In [6]:
import openpyxl
with pd.ExcelWriter("ml_equity_premium_results.xlsx", engine='openpyxl', mode='a') as writer:
    market_timing_perf_df.to_excel(writer, sheet_name='market_timing_performance')
market_timing_perf_df

,Average return(%),Sharpe ratio
Buy-and-hold,11.150175,0.135182
PLS,14.682368,0.387562
PCR,19.030842,0.516624
LASSO,22.410117,0.627766
ENet,22.223671,0.628025
RF,21.394123,0.627924
NN2,21.981387,0.632627
NN4,19.038211,0.542786
Ridge,21.511979,0.589083
HA,25.627510,0.800440
